In [ ]:
# 1. Instalar o uv para gerenciar dependências
!pip install uv --quiet

# 2. Usa o uv para resolver e instalar o PYield respeitando o ambiente do Colab
!uv pip install PYield

# Importar a biblioteca PYield
import pyield as yd
from pyield import bday

print("PYield version:", yd.__version__)

In [ ]:
# Importar a biblioteca PYield
import pyield as yd

yd.__version__

In [ ]:
# 1. Contagem de dias úteis
# A função `bday.count` calcula o número de dias úteis entre duas datas usando
# o calendário de dias úteis do Brasil. Formatos aceitos: `DD-MM-YYYY` ou `YYYY-MM-DD`.
du = bday.count("02-01-2024", "09-01-2024")
print("Dias úteis:", du)
# Lista de vários períodos (retorna Series Polars)
bday.count("01-01-2024", ["08-01-2024", "22-01-2024"])

In [ ]:
# 2. Offset de datas para cair em dias úteis.
# `bday.offset(data, n)` ajusta para o próximo dia útil considerando feriados.
ajustada = bday.offset("20-11-2024", 0)  # 20/11/2024 é feriado nacional recente
print("Data ajustada:", ajustada)

In [ ]:
# 2. A contagem de dias úteis e de offset são funções vetorizadas do Polars.
# Aceitam listas ou Series do Polars ou Pandas.
# Mas sempre retornam Series do Polars.
datas = ["15-11-2023", "20-11-2024", "25-12-2023"]
offsets = [0, 0, 2]
bday.offset(datas, offsets)

In [ ]:
# Vamos ajustar os vencimentos das LTN para coindirem com dias úteis.
# Primeiro, obtemos os dados das LTN. Vamos manter somente as colunas necessárias.
df_ltn = yd.ltn.data(date="27-11-2025").select(
    data_ref="ReferenceDate",
    titulo="BondType",
    vencimento="MaturityDate",
    taxa="IndicativeRate",
)
df_ltn

In [ ]:
# Agora, aplicamos o offset para ajustar os vencimentos.
vencimentos_ajustados = bday.offset(df_ltn["vencimento"], 0)
df_ltn = df_ltn.with_columns(
    vencimentos_ajustados.alias("vencimento_ajustado"),
)
df_ltn

In [ ]:
# Vamos calcular o DU entre a data de referência e o vencimento.
# Tanto faz usar o vencimento original ou o ajustado, pois a contagem só considera DU.
du_vencimento = bday.count(df_ltn["data_ref"], df_ltn["vencimento"])
du_vencimento_ajustado = bday.count(df_ltn["data_ref"], df_ltn["vencimento_ajustado"])
df_ltn = df_ltn.with_columns(
    du_vencimento.alias("du"),
    du_vencimento_ajustado.alias("du_ajustado"),
)
df_ltn

In [ ]:
# Podemos calcular a taxa forward entre os vértices das LTNs dado que essas taxas já
# são spot (as LTN não pagam cupom).
# Esse cálculo é também vetorizado, aceitando Series do Polars ou Pandas.
tx_forward = yd.forwards(bdays=df_ltn["du"], rates=df_ltn["taxa"])
df_ltn = df_ltn.with_columns(
    tx_forward.alias("taxa_fwd"),
)
df_ltn


In [ ]:
# 2. Futuros DI (B3)
# Obtém dados de contratos futuros de DI1 (taxas de vencimento).
# Faz requisição à B3, portanto é necessário acesso à internet.
df = yd.futures(date="26-12-2023", contract_code="DI1")
df

In [ ]:
# Vamos usar as taxas de ajuste do DI1 como base na geração de taxas interpoladas.
df_di1 = df.select(
    data_ref="TradeDate",
    vencimento="ExpirationDate",
    taxa_ajuste="SettlementRate",
    du="BDaysToExp",
)
# Classe de interpolação da PYield
flat_fwd = yd.Interpolator(
    method="flat_forward",
    known_bdays=df_di1["du"],
    known_rates=df_di1["taxa_ajuste"],
)
flat_fwd

In [ ]:
# Podemos agora calcular taxas interpoladas para qualquer DU.
# Se du é conhecido, retorna a taxa existente.
# Se du não é conhecido, retorna a taxa interpolada (flat forward no caso).
du_alvo = 4
print("DU existente  :", flat_fwd.interpolate(du_alvo))  # DU existente
du_alvo = 10
print("DU interpolado:", flat_fwd.interpolate(du_alvo))  # DU interpolado

In [ ]:
# Converter para Pandas (se desejar interoperar com outras libs):
df_pandas = df.to_pandas(use_pyarrow_extension_array=True)
df_pandas

## 3. Próximos Passos
- Explorar módulos de títulos públicos (`pyield.ntnb`) e índices (`pyield.ipca`).
- Buscar dados históricos de leilões do Tesouro Nacional.

Consulte a documentação para mais detalhes.